In [1]:
# !pip install --upgrade pip
# !pip install -q arch statsmodels pmdarima

# Volatility benchmarks (classic models)
Compare traditional volatility forecasters against the current best model saved under `artifacts/best_overall_model`.

### Plan (following the xgboost notebook style)
- Use the same train/val/test splits from `artifacts/data` (0_data_prep).
- Fit classic vol models on those splits: GARCH(1,1), HAR-RV (linear), ARIMA on log-vol, and a simple Ridge/ElasticNet on Level-1 features.
- Compute validation RMSE (primary) and MAE/R² (aux) and compare to `best_overall_model` (currently xgb_tuned).
- Keep the test split untouched until a winner is chosen; then optionally refit on train+val and score test once.

In [2]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA

In [3]:
# paths to artifacts (same path dance as other notebooks)
_start = Path.cwd().resolve()
_candidates = [_start] + list(_start.parents)
_repo = None
for p in _candidates:
    if (p / 'notebooks/model_evaluation_final/artifacts/data').exists():
        _repo = p
        break
REPO_ROOT = _repo if _repo else _start
ARTIFACTS_DIR = REPO_ROOT / 'notebooks/model_evaluation_final/artifacts'
DATA_DIR = ARTIFACTS_DIR / 'data'
BEST_OVERALL_DIR = ARTIFACTS_DIR / 'best_overall_model'
BEST_INDIVIDUAL_DIR = ARTIFACTS_DIR / 'best_individual_model'

ARTIFACTS_DIR, DATA_DIR, BEST_OVERALL_DIR

(PosixPath('/Users/aayushrijal/Documents/GitHub/volatility_forecast/notebooks/model_evaluation_final/artifacts'),
 PosixPath('/Users/aayushrijal/Documents/GitHub/volatility_forecast/notebooks/model_evaluation_final/artifacts/data'),
 PosixPath('/Users/aayushrijal/Documents/GitHub/volatility_forecast/notebooks/model_evaluation_final/artifacts/best_overall_model'))

In [4]:
# load data artifacts from 0_data_prep output (same split as xgboost)
with open(DATA_DIR / 'feature_columns.json') as f:
    feature_cols = json.load(f)
with open(DATA_DIR / 'split_config.json') as f:
    split_cfg = json.load(f)

X_train = pd.read_parquet(DATA_DIR / 'X_train.parquet')
X_val = pd.read_parquet(DATA_DIR / 'X_val.parquet')
y_train = pd.read_parquet(DATA_DIR / 'y_train.parquet')[split_cfg['target_col']].values
y_val = pd.read_parquet(DATA_DIR / 'y_val.parquet')[split_cfg['target_col']].values
has_test = (DATA_DIR / 'y_test.parquet').exists()
X_test = pd.read_parquet(DATA_DIR / 'X_test.parquet') if has_test else None
y_test = pd.read_parquet(DATA_DIR / 'y_test.parquet')[split_cfg['target_col']].values if has_test else np.array([])

print('Loaded data from artifacts/data:')
print('  X_train shape:', X_train.shape)
print('  X_val shape  :', X_val.shape)
print('  y_train len  :', len(y_train))
print('  y_val len    :', len(y_val))
print('  has_test     :', has_test)

Loaded data from artifacts/data:
  X_train shape: (3154, 20)
  X_val shape  : (788, 20)
  y_train len  : 3154
  y_val len    : 788
  has_test     : True


### Candidate models
- **GARCH(1,1)** on returns (baseline).
- **HAR-RV**: linear regression on realized vol windows (e.g., 5d/20d/60d) plus optional VIX term.
- **ARIMA/ARFIMA** on log-vol or realized volatility (for long memory).
- **Ridge/ElasticNet** on the existing Level-1 features as a simple regularized baseline.

Use the same train/val split for all, with RMSE on validation as the selection metric (MAE/R² for color).

In [ ]:
# helpers
def compute_metrics(y_true, y_pred):
    return {
        'rmse': float(np.sqrt(mean_squared_error(y_true, y_pred))),
        'mae': float(mean_absolute_error(y_true, y_pred)),
        'r2': float(r2_score(y_true, y_pred))
    }

results = []  # collect {'name': ..., 'val': {...}, 'test': {...}, 'notes': str}


def add_result(name, val_pred, test_pred=None, notes=None):
    # Replace any existing entry with the same name to avoid duplicates across reruns
    global results
    results = [r for r in results if r.get('name') != name]
    entry = {'name': name, 'val': compute_metrics(y_val, val_pred)}
    if test_pred is not None and len(test_pred) == len(y_test) and len(y_test) > 0:
        entry['test'] = compute_metrics(y_test, test_pred)
    if notes:
        entry['notes'] = notes
    results.append(entry)


# Utility: get current best_overall benchmark (for later comparison)
with open(BEST_OVERALL_DIR / 'metrics.json') as f:
    best_overall = json.load(f)
benchmark_name = best_overall['best_overall']['name']
benchmark_val_rmse = best_overall['best_overall']['val']['rmse']
benchmark_test_rmse = best_overall['best_overall'].get('test', {}).get('rmse') if isinstance(best_overall['best_overall'].get('test'), dict) else None
print('Benchmark (best_overall):', benchmark_name, 'val RMSE=', benchmark_val_rmse, 'test RMSE=', benchmark_test_rmse)

Benchmark (best_overall): xgb_tuned val RMSE= 0.00907279671175083 test RMSE= None


In [15]:
# GARCH(1,1) directly on target series (rv_5d already log-scale)
try:
    scale = 100.0  # rescale to improve optimizer convergence
    series = y_train.flatten() * scale
    am = arch_model(series, vol='Garch', p=1, o=0, q=1, dist='normal', rescale=False)
    res = am.fit(disp='off')
    val_var = res.forecast(horizon=len(y_val)).variance.values[-1]
    test_var = res.forecast(horizon=len(y_test)).variance.values[-1] if has_test and len(y_test) > 0 else None
    val_fc = np.sqrt(val_var) / scale
    test_fc = (np.sqrt(test_var) / scale) if test_var is not None else None
    add_result('garch_1_1', val_pred=val_fc, test_pred=test_fc, notes='arch GARCH(1,1) on rv_5d target, rescaled x100')
    print('GARCH(1,1) fitted on target and added (rescaled x100).')
except Exception as e:
    print('GARCH(1,1) failed:', e)

GARCH(1,1) fitted on target and added (rescaled x100).


In [16]:
# HAR-RV style linear regression on realized vol windows
har_feats = [c for c in ['spy_vol_5d', 'spy_vol_20d', 'spy_vol_60d', 'vix_term'] if c in X_train.columns]
if len(har_feats) == 0:
    print('No HAR features found; skipping HAR-RV linear.')
else:
    try:
        lr = LinearRegression()
        lr.fit(X_train[har_feats], y_train)
        val_pred = lr.predict(X_val[har_feats])
        test_pred = lr.predict(X_test[har_feats]) if has_test and X_test is not None else None
        add_result('har_rv_linear', val_pred=val_pred, test_pred=test_pred, notes=f'HAR on {har_feats}')
        print('HAR-RV linear fitted and added.')
    except Exception as e:
        print('HAR-RV linear failed:', e)

HAR-RV linear fitted and added.


In [17]:
# ARIMA on target (rv_5d already in log-scale form)
try:
    series = y_train.flatten()
    model = ARIMA(series, order=(1, 0, 1))
    res = model.fit()
    val_fc = res.forecast(steps=len(y_val))
    val_pred = val_fc
    test_pred = None
    if has_test and len(y_test) > 0:
        test_fc = res.forecast(steps=len(y_val) + len(y_test))[-len(y_test):]
        test_pred = test_fc
    add_result('arima_target', val_pred=val_pred, test_pred=test_pred, notes='ARIMA(1,0,1) on rv_5d target')
    print('ARIMA on target fitted and added.')
except Exception as e:
    print('ARIMA target failed:', e)

ARIMA on target fitted and added.


In [18]:
# Ridge/ElasticNet skipped here (already evaluated in individual models)
print('Skipping Ridge/ElasticNet: already evaluated in individual model sweeps.')

Skipping Ridge/ElasticNet: already evaluated in individual model sweeps.


In [ ]:
# summarize and compare vs benchmark
if len(results) == 0:
    print('No candidate results yet. Run the modeling cells above first.')
else:
    # de-duplicate by candidate name (keep last occurrence)
    unique = {}
    for r in results:
        unique[r['name']] = r
    comparison = []
    for r in unique.values():
        row = {
            'candidate': r['name'],
            'val_rmse': r['val']['rmse'],
            'val_mae': r['val']['mae'],
            'val_r2': r['val']['r2'],
            'test_rmse': r.get('test', {}).get('rmse') if isinstance(r.get('test'), dict) else None,
            'notes': r.get('notes')
        }
        comparison.append(row)
    comp_df = pd.DataFrame(comparison).sort_values('val_rmse')
    print('Benchmark val RMSE (best_overall):', benchmark_val_rmse)
    print('Validation RMSE by candidate:')
    for _, row in comp_df.iterrows():
        print(f"  {row['candidate']}: {row['val_rmse']}")
    winner = comp_df.iloc[0]
    print('Top candidate:', winner['candidate'], 'val RMSE=', winner['val_rmse'])
    comp_df

Benchmark val RMSE (best_overall): 0.00907279671175083
Validation RMSE by candidate:
  har_rv_linear: 0.010295707231905898
  har_rv_linear: 0.010295707231905898
  arima_target: 0.011956471283582628
  arima_target: 0.011956471283582628
  garch_1_1: 0.013196778419573833
  garch_1_1: 3.2083032074773485
Top candidate: har_rv_linear val RMSE= 0.010295707231905898


### Next steps
- Install required libs for the models you want to run (e.g., `arch`, `statsmodels`, `pmdarima`, `scikit-learn`).
- Fill the TODO blocks to fit each candidate, call `add_result(...)`, and re-run the comparison cell.
- If a candidate beats the benchmark on validation RMSE, optionally refit it on train+val and score once on test for final reporting.

### Notes
- Uses the same train/val splits as XGBoost (`artifacts/data`).
- Runs lightweight models without new dependencies where possible (Ridge/ElasticNet/HAR). GARCH/ARIMA try to run if `arch`/`statsmodels` are installed; otherwise they are skipped gracefully.
- Selection metric: validation RMSE (MAE/R² for color). Test is computed only if available and the model supports it.

In [12]:
# Install dependencies for classic volatility models
# (Run once per environment/kernel)
!pip install -q arch statsmodels pmdarima